In [ ]:
'''
This script looks at population dist. of lifetime image response sparsity. 
It is specifically designed to compare conditions (A vs Not A, Hit vs. Miss)
Note that I have not removed the profain variable names, nor do I intend to. 
Created by Yoni Browning, August 2018
'''

In [1]:
#%load_ext autoreload
#%autoreload 2
import matplotlib as mpl
%matplotlib inline
import scipy as sp
import numpy as np
import pandas as pd
# sometimes order maters on these
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import oBehave.behavior_analysis as obba
import oBehave.plotting_stuff as obps
import oBehave.helperfunctions as obhf
import oBehave.neural_analysis as obna

import sklearn.decomposition as decomp


from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
from visual_behavior.ophys.response_analysis.utilities import get_trace_around_timepoint,get_nearest_frame
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis

experiment_id = 639438856


/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load all the data we will want for this
manifest = obhf.load_manifest()
obba.includeNovelSession(manifest)
dataset= VisualBehaviorOphysDataset(experiment_id=experiment_id, cache_dir=obhf.drive_path)
#analysis = ResponseAnalysis(dataset)

In [ ]:
def compute_lifetime_sparseness(image_responses):
    # image responses should be an array of the trial averaged responses to each image
    # sparseness = 1-(sum of trial averaged responses to images / N)squared / (sum of (squared mean responses / n)) / (1-(1/N))
    # N = number of images
    # Modified shamelessly from allen institute code, s.t. we now include an abs in numerator.
    N = float(len(image_responses))
    ls = ((1-(1/N) * ((np.power(np.abs(image_responses).sum(axis=0),2)) / (np.power(image_responses,2).sum(axis=0)))) / (1-(1/N)))
    return ls

# compute sparsity for 1st, second, etc. flashes for each cell
# also Average, and hits vs misses.
keylist = ['experiment_id','cell_indices','cell_specimen_ids','session_type','cre_line',
           'SparAve','Spar1','Spar2','Spar3','Spar4','Spar5','Shit','Smiss']
cr = {k:[] for k in keylist}
for ex,row in manifest.iterrows():
    if not row['first_session']:
        continue
    print(row['experiment_id'])
    dataset= VisualBehaviorOphysDataset(experiment_id=row['experiment_id'], cache_dir=obhf.drive_path)
    analysis = ResponseAnalysis(dataset)
    FR = obba.include_add_repeats(analysis.flash_response_df,dataset.all_trials)
    unique_cells = np.unique(FR['cell'])
    unique_images = np.unique(FR['image_name'])
    for ii in unique_cells:
        cr['experiment_id'].append(row['experiment_id'])
        cr['cell_indices'].append(dataset.cell_indices[ii])
        cr['cell_specimen_ids'].append(dataset.cell_specimen_ids[ii])
        cr['cre_line'].append(row['cre_line'])
        cr['session_type'].append(row['session_type'])
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['SparAve'].append(compute_lifetime_sparseness(im_means))
        for rr in [1,2,3,4,5]:
            im_means = [FR[(FR['image_name']==image)&(FR['cell']==ii)&(FR['repeats']==rr)].mean_response.mean() for image in np.unique(FR['image_name'])]
            cr['Spar'+str(rr)].append(compute_lifetime_sparseness(im_means))
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)&(FR['repeats']==1)&(FR['response_type']=='HIT')].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['Shit'].append(compute_lifetime_sparseness(im_means))
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)&(FR['repeats']==1)&(FR['response_type']=='MISS')].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['Smiss'].append(compute_lifetime_sparseness(im_means))    
CellResp = pd.DataFrame(cr)
CellResp.to_pickle('./popMeanResponseSelectivity_rerun.pkl')


644942849
loading trial response dataframe
loading flash response dataframe
645035903
loading trial response dataframe
loading flash response dataframe
645086795
loading trial response dataframe
loading flash response dataframe
645362806
loading trial response dataframe
loading flash response dataframe
646922970
loading trial response dataframe
loading flash response dataframe
647108734
loading trial response dataframe
loading flash response dataframe
647551128
loading trial response dataframe
loading flash response dataframe
647887770
loading trial response dataframe
loading flash response dataframe
652844352
loading trial response dataframe
loading flash response dataframe
653053906
loading trial response dataframe
loading flash response dataframe
653123781
loading trial response dataframe
loading flash response dataframe
639253368
loading trial response dataframe
loading flash response dataframe
639438856
loading trial response dataframe
loading flash response dataframe
639769395
lo

In [ ]:
# plot some histograms!
plt.hist(CellResp['Shit'][np.isfinite(CellResp['Shit'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type']=='behavior_session_A')],bins = 50,alpha = .5,normed = True);
plt.hist(CellResp['Smiss'][np.isfinite(CellResp['Smiss'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type'] == 'behavior_session_A')],bins = 50,alpha = .5,normed = True);
plt.legend(['Hits (1st flash)','Misses (1st Flash)'])
plt.figure()
plt.hist(CellResp['Shit'][np.isfinite(CellResp['Shit'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type']=='behavior_session_A')],histtype='step',normed=True,cumulative=True)
#plt.hist(CellResp['Spar2'][np.isfinite(CellResp['Spar2'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type'] == 'behavior_session_B')],bins = 50,alpha = .5,normed = True);

#plt.hist(CellResp['Spar2'][np.isfinite(CellResp['Spar2'])&(CellResp['cre_line']=='Vip-IRES-Cre')&(CellResp['session_type']=='behavior_session_A')],bins = 20,alpha = .5,normed = True);
#plt.hist(CellResp['Spar2'][np.isfinite(CellResp['Spar2'])&(CellResp['cre_line']=='Vip-IRES-Cre')&(CellResp['session_type'] == 'behavior_session_C')],bins = 20,alpha = .5,normed = True);
#sp.stats.ks_2samp(CellResp['SparAve'][np.isfinite(CellResp['SparAve'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type']=='behavior_session_A')],
#                  CellResp['SparAve'][np.isfinite(CellResp['SparAve'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type'] == 'behavior_session_D')])


In [ ]:
# confirm that the dataframe is sensable.
CellResp.head()

In [ ]:
# funky function for make cumulativy density plots. There is a more general version
# of this in the MakeSparsityPlots script.
def ecdf(data,plotme = True,ax = None,lineopts={'linestyle':'-','marker':'.'}):
    if ax is None:
        ax = plt.gca()
    data = np.array(data)# Just in case
    data = data[np.isfinite(data)]
    cdfx = np.sort(np.unique(data[np.isfinite(data)]))
    yval = np.zeros(cdfx.shape)
    for ii,xx in enumerate(cdfx):
        yval[ii] = float(len(data[data<xx]))/float(len(data))
    if plotme:
        ax.plot(cdfx,yval,**lineopts)
    return cdfx,yval

#ecdf(CellResp['Spar1'].values);
ecdf(CellResp['SparAve'][np.isfinite(CellResp['SparAve'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type']=='behavior_session_A')].values);
ecdf(CellResp['SparAve'][np.isfinite(CellResp['SparAve'])&(CellResp['cre_line']=='Slc17a7-IRES2-Cre')&(CellResp['session_type']=='behavior_session_B')].values);


In [ ]:
# make plots with ecdf. User will need to specify which things they want to compare
CR = CellResp.copy()
CR = CR.merge(manifest,left_on='experiment_id',right_on='experiment_id',how='left')
# plt.figure()
# ecdf(CR['SparAve'][np.isfinite(CR['SparAve'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y']=='behavior_session_A')&(CR['targeted_structure']=='VISp')].values);
# ecdf(CR['SparAve'][np.isfinite(CR['SparAve'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y']=='behavior_session_A')&(CR['targeted_structure']=='VISal')].values);
# plt.legend(['VISp','VISal'])

fig = plt.figure(tight_layout = True)
axes = fig.subplots(1,2)
axes[0].hist(CR['Spar1'][np.isfinite(CR['Spar1'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y'] is not 'behavior_session_A')&(CR['targeted_structure']=='VISp')].values,bins =30,normed = True,alpha = .6)
axes[0].hist(CR['Spar2'][np.isfinite(CR['Spar2'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y'] is not'behavior_session_A')&(CR['targeted_structure']=='VISp')].values,bins =30,normed = True,alpha = .6)
axes[0].hist(CR['Spar3'][np.isfinite(CR['Spar3'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y'] is not'behavior_session_A')&(CR['targeted_structure']=='VISp')].values,bins =30,normed = True,alpha = .6)
axes[0].legend(['Flash 1','2','3'])
axes[0].set_title('Distribution')
axes[0].set_ylabel('% Pop/Bin')
axes[0].set_xlabel('Sparsity')

ecdf(CR['Spar1'][np.isfinite(CR['Spar1'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y'] is not 'behavior_session_A')&(CR['targeted_structure']=='VISp')].values,ax = axes[1]);
ecdf(CR['Spar2'][np.isfinite(CR['Spar2'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y'] is not'behavior_session_A')&(CR['targeted_structure']=='VISp')].values,ax = axes[1]);
ecdf(CR['Spar3'][np.isfinite(CR['Spar3'])&(CR['cre_line_x']=='Slc17a7-IRES2-Cre')&(CR['session_type_y'] is not 'behavior_session_A')&(CR['targeted_structure']=='VISp')].values,ax = axes[1]);
axes[0].legend(['Flash 1','2','3'])
axes[1].set_title('Emperical CDF')
axes[1].set_ylabel('Fraction Pop')
axes[1].set_xlabel('Sparsity')



